R에는 데이터프레임과 문자열 기호를 이용하여 회귀모형을 정의하는 방법이 존재한다. statsmodel 패키지에서도 이러한 모형 정의 방법을 지원한다.

## patsy
- 회귀분석 전처리를 위한 패키지로 데이터프레임을 가공하여 인코딩, 변환 등을 쉽게 해주는 기능을 제공한다.
- 실험설계행렬(experiment design matrix)을 간단하게 만들 수 있다.
- `data_transformed = dmatrix(formula, data)
`

In [1]:
from patsy import dmatrix

In [3]:
import numpy as np
import pandas as pd
np.random.seed(0)
x1 = np.random.rand(5) + 10
x2 = np.random.rand(5) * 10
y = x1 + 2 * x2 + np.random.randn(5)
df = pd.DataFrame(np.array([x1,x2,y]).T,columns=['x1','x2','y'])
df

,x1,x2,y
0,10.548814,6.458941,23.610739
1,10.715189,4.375872,20.921207
2,10.602763,8.917730,29.199261
3,10.544883,9.636628,29.939813
4,10.423655,3.834415,18.536348


dmatrix의 첫 번째 기능은 automated augmentation이다. 대상이 되는 데이터에 자동으로 intercept라는 이름의 상수항이 들어있는 컬럼을 추가한다.

In [4]:
dmatrix('x1',df)

DesignMatrix with shape (5, 2)
  Intercept        x1
          1  10.54881
          1  10.71519
          1  10.60276
          1  10.54488
          1  10.42365
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)

## R-style formula operation
- 
기호	설명
- `+`	설명 변수 추가
- `-`	설명 변수 제거
- `1, 0`	intercept. (제거시 사용)
- `:`	interaction (곱)
- `*` a*b = a + b + a:b
- `/`	a/b = a + a:b
- `~`	종속 - 독립 관계

In [78]:
from sklearn.datasets import load_boston
import statsmodels.api as sm
boston = load_boston()
dfX0_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
dfX_boston = sm.add_constant(dfX0_boston)
dfy_boston = pd.DataFrame(boston.target, columns=["MEDV"])
df_boston = pd.concat([dfX_boston, dfy_boston], axis=1)
df_boston.tail()

,const,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
501,1.0,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,1.0,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,1.0,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,1.0,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
505,1.0,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,11.9


In [76]:
model = sm.OLS.from_formula("MEDV ~ CRIM+ ZN + INDUS + C(CHAS) + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + 0", data=df_boston)
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sat, 30 Jun 2018   Prob (F-statistic):          6.95e-135
Time:                        11:45:07   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
C(CHAS)[0.0]    36.4911      5.104      7.149   

In [77]:
model = sm.OLS.from_formula("MEDV ~ CRIM+ ZN + INDUS + C(CHAS) + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT ", data=df_boston)
print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sat, 30 Jun 2018   Prob (F-statistic):          6.95e-135
Time:                        11:58:09   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         36.4911      5.104      7.